### Import data

In [5]:
include("importData.jl")
include("DDEmodel.jl")
include("plot.jl")

# import data

# lapatinib
conc_l, pop_l, g2_l, g1_l, g1_0_l, g2_0_l = setup_data("lapatinib");


┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, ::Colon, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = get_data(::String, ::String) at importData.jl:29
└ @ Main /home/farnazm/dde/lapatinib/DrugResponseModel.jl/src/importData.jl:29
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, ::Colon, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = get_data(::String, ::String) at importData.jl:37
└ @ Main /home/farnazm/dde/lapatinib/DrugResponseModel.jl/src/importData.jl:37


### Estimate 

In [6]:
# initial guess
initial_guess  = [0.0596124, 0.0331098, 0.005, 0.005, 0.0596124, 0.0331098, 0.005, 0.005,0.0596124, 0.0331098, 0.005, 0.005, 0.0596124, 0.0331098, 0.005, 0.005, 0.0596124, 0.0331098, 0.005, 0.005, 0.0596124, 0.0331098, 0.005, 0.005, 0.0596124, 0.0331098, 0.005, 0.005, 0.0596124, 0.0331098, 0.005, 0.005, 25.0, 7.0]

# bounds 
lower_bnd = -8.0*ones(34)
upper_bnd = 1.0*ones(32)
append!(upper_bnd, [30.0, 20.0])
# max number of steps
maxSteps = 1e4
parameters = zeros(34)
j = 4
# [initial_guess[33], initial_guess[34]]
# _, parameters = optimization(g1_l, g2_l, g1_0_l, g2_0_l, initial_guess, lower_bnd, upper_bnd, maxSteps)
DDEsolve(initial_guess, g1_l, g2_l, g1_0_l, g2_0_l, 4)

hello


MethodError: MethodError: no method matching __solve(::DDEProblem{Array{Float64,1},Tuple{Float64,Float64},Array{Float64,1},Tuple{},true,Array{Float64,1},DDEFunction{true,typeof(DDEmodel),LinearAlgebra.UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing},getfield(Main, Symbol("#h#46")){Array{Float64,1},Array{Float64,1}},Nothing})
Closest candidates are:
  __solve(::DiffEqBase.AbstractDDEProblem, !Matched::DelayDiffEq.AbstractMethodOfStepsAlgorithm, !Matched::Any...; kwargs...) at /home/farnazm/.julia/packages/DelayDiffEq/ypXUT/src/solve.jl:4
  __solve(!Matched::DiffEqBase.AbstractEnsembleProblem, !Matched::Union{Nothing, DiffEqBase.DEAlgorithm}; kwargs...) at /home/farnazm/.julia/packages/DiffEqBase/LCorD/src/ensemble/basic_ensemble_solve.jl:27
  __solve(!Matched::DiffEqBase.AbstractEnsembleProblem, !Matched::Union{Nothing, DiffEqBase.DEAlgorithm}, !Matched::DiffEqBase.BasicEnsembleAlgorithm; trajectories, batch_size, pmap_batch_size, kwargs...) 

In [7]:
p1 = plotIt(parameters[:, 1], 1, "control", :left)
p2 = plotIt(parameters[:, 2], 2, "conc. = 5 nM", false)
p3 = plotIt(parameters[:, 3], 3, "conc. = 5 nM", false)
p4 = plotIt(parameters[:, 4], 4, "conc. = 5 nM", false)
p5 = plotIt(parameters[:, 5], 5, "conc. = 5 nM", false)
p6 = plotIt(parameters[:, 6], 6, "conc. = 5 nM", false)
p7 = plotIt(parameters[:, 7], 7, "conc. = 5 nM", false)
p8 = plotIt(parameters[:, 8], 8, "conc. = 5 nM", false)
plot(p1, p2, p3, p4, p5, p6, p7, p8, layout = (2,4))
plot!(size = (1400, 700), dpi = 150)
ylims!((0.0, 100.0))


┌ Warning: dt <= dtmin. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ DiffEqBase /home/farnazm/.julia/packages/DiffEqBase/LCorD/src/integrator_interface.jl:156


BoundsError: BoundsError: attempt to access 1-element Array{Float64,1} at index [2]